In [3]:
#Imports
import numpy as np
import pandas as pd

from tqdm import tqdm

In [4]:
#Data loading 

usefull_columns = ['genres','keywords','popularity']
#TO BE REMOVED to let the user give the array as a parameter of the function. 
Movies = pd.read_csv("../../Datasets/Transformed.csv",usecols=usefull_columns)

#We reduce the dataframe size with only the usefull column to earn some computation time.

In [5]:
#Constant definition

#Cost added if the first genre is similar between two films
first_genre = 5
#Cost added if the secondary genre is similar between two films
second_genre = 1
#Cost added by similar keyword identical between two films
keyword_cost = 1

In [11]:
def cost_function():
    """Define the cost between the film given in index and the others one."""
    costs = np.zeros([Movies.shape[0],Movies.shape[0]])
    
    for i in tqdm(range(Movies.shape[0])):
        current_film = Movies.iloc[i]
        genres_current = get_genres(current_film)
        kw_current = get_keywords(current_film)
        popularity_current = current_film['popularity']

        for j in range(i,Movies.shape[0]):
            cost = 0

            b_film = Movies.iloc[j]
            genres_b = get_genres(b_film)
            popularity_b = b_film['popularity']
            #First we only select the first genre to determine the similarity because it's more important that the other genre.
            if len(genres_current) > 0  & len(genres_b) > 0:
                if (genres_current[0] == genres_b[0]):
                    cost += first_genre

                #This give us the number of similar genres. We pop the first one because we already compare them.
                cost += np.sum(np.in1d(genres_current,genres_b.pop(0),assume_unique='True')) * second_genre


            kw_b = get_keywords(b_film)
            #This give us the number of similar keywords.
            cost += np.sum(np.in1d(kw_current,kw_b,assume_unique='True')) * keyword_cost
            
            
            #impossible here because we ignore to much popularity
            #cost = (cost * popularity_b/100) / (popularity_current/100)
            costs[i,j] = cost
            costs[j,i] = cost
            
            
    return costs

def get_genres(film):
    genres = str(film['genres'])
    if genres == 'nan':
        return[]
    else:
        genres = genres.split(",")
    return genres

def get_keywords(film):
    kw = str(film['keywords'])
    if kw == 'nan':
        return[]
    else:
        kw = kw.split(",")
    return kw

In [12]:
costs = cost_function()

#Save the costs
np.savez_compressed("../../Datasets/costs.npz", costs, delimiter=",")
print(costs)

100%|██████████| 4809/4809 [48:27<00:00,  1.65it/s]  


[[  0.   0.   0. ...,   0.   0.   0.]
 [  0.  17.   0. ...,   0.   0.   0.]
 [  0.   0.   8. ...,   0.   0.   0.]
 ..., 
 [  0.   0.   0. ...,  10.   0.   0.]
 [  0.   0.   0. ...,   0.  11.   0.]
 [  0.   0.   0. ...,   0.   0.  16.]]


In [42]:
#Save the costs
np.savez_compressed("../../Datasets/costs.npz", costs, delimiter=",")